In [ ]:
# ==================== KAGGLE ENVIRONMENT SETUP ====================
# Run this cell first in your Kaggle notebook

# Install required packages
!pip install -q seaborn  # For advanced plotting (might need update)
!pip install -q tqdm     # For progress bars (usually pre-installed)

# Check if key packages are available (most should be pre-installed in Kaggle)
import sys

# Check Python version
print(f"Python version: {sys.version}")

# Test imports and show versions
try:
    import torch
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"   CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   GPU count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
except ImportError:
    print("❌ PyTorch not available")

try:
    import torchvision
    print(f"✅ TorchVision: {torchvision.__version__}")
except ImportError:
    print("❌ TorchVision not available")

try:
    import cv2
    print(f"✅ OpenCV: {cv2.__version__}")
except ImportError:
    print("❌ OpenCV not available - installing...")
    !pip install opencv-python
    import cv2
    print(f"✅ OpenCV installed: {cv2.__version__}")

try:
    import numpy as np
    print(f"✅ NumPy: {np.__version__}")
except ImportError:
    print("❌ NumPy not available")

try:
    import pandas as pd
    print(f"✅ Pandas: {pd.__version__}")
except ImportError:
    print("❌ Pandas not available")

try:
    from PIL import Image
    print(f"✅ Pillow (PIL): Available")
except ImportError:
    print("❌ Pillow not available")

try:
    import matplotlib.pyplot as plt
    import matplotlib
    print(f"✅ Matplotlib: {matplotlib.__version__}")
except ImportError:
    print("❌ Matplotlib not available")

try:
    import seaborn as sns
    print(f"✅ Seaborn: {sns.__version__}")
except ImportError:
    print("❌ Seaborn not available - should be installed above")

try:
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    import sklearn
    print(f"✅ Scikit-learn: {sklearn.__version__}")
except ImportError:
    print("❌ Scikit-learn not available")

try:
    from tqdm import tqdm
    print(f"✅ TQDM: Available")
except ImportError:
    print("❌ TQDM not available - should be installed above")

# Test basic functionality
print("\n🔍 Testing Basic Functionality:")

# Test file system access
import os
if os.path.exists("/kaggle/input"):
    print("✅ Kaggle input directory accessible")
    if os.path.exists("/kaggle/input/znz-cloves"):
        print("✅ Dataset found at /kaggle/input/znz-cloves")
        
        # List dataset contents
        print("\n📁 Dataset Structure:")
        for item in os.listdir("/kaggle/input/znz-cloves"):
            item_path = os.path.join("/kaggle/input/znz-cloves", item)
            if os.path.isdir(item_path):
                count = len([f for f in os.listdir(item_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))])
                print(f"   📂 {item}: {count} images")
            else:
                print(f"   📄 {item}")
    else:
        print("❌ Dataset not found - make sure it's uploaded as 'znz-cloves'")
else:
    print("❌ Kaggle input directory not found")

if os.path.exists("/kaggle/working"):
    print("✅ Kaggle working directory accessible")
else:
    print("❌ Kaggle working directory not found")

# Test GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    # Test GPU memory
    try:
        test_tensor = torch.randn(100, 100).to(device)
        print(f"✅ GPU memory test passed")
        print(f"   Allocated: {torch.cuda.memory_allocated()/1024**2:.1f} MB")
        print(f"   Cached: {torch.cuda.memory_reserved()/1024**2:.1f} MB")
        del test_tensor
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"❌ GPU test failed: {e}")
else:
    print("❌ CUDA not available")

print("\n" + "="*60)
print("🚀 ENVIRONMENT SETUP COMPLETE!")
print("="*60)

# Set random seeds for reproducibility
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("🎯 Random seeds set for reproducible results")
print("📝 You can now run the main pipeline code!")

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import zipfile
import shutil
import json
from collections import defaultdict, Counter
import random
from datetime import datetime

# Set up Kaggle environment
print("🚀 Context-Aware Pipeline for Grade Classification")
print("Environment: Kaggle with T4 x2 GPU")
print("=" * 60)

# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU Count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# ==================== STEP 1: DATASET EXPLORATION AND ANALYSIS ====================

class DatasetExplorer:
    def __init__(self, base_path="/kaggle/input/znz-cloves"):
        self.base_path = base_path
        self.grade_paths = {
            'Grade 1': f"{base_path}/Grade 1",
            'Grade 2': f"{base_path}/Grade 2", 
            'Grade 3': f"{base_path}/Grade 3",
            'Grade 4': f"{base_path}/Grade 4"
        }
        self.group_paths = {
            'Group Grade 1': f"{base_path}/group_photos/grade1",
            'Group Grade 2': f"{base_path}/group_photos/grade2",
            'Group Grade 3': f"{base_path}/group_photos/grade3",
            'Group Grade 4': f"{base_path}/group_photos/grade4"
        }
        self.all_paths = {**self.grade_paths, **self.group_paths}
        self.dataset_stats = {}
    
    def explore_dataset(self):
        """Comprehensive dataset exploration with visualizations and statistics"""
        print("📊 STEP 1: Dataset Exploration and Analysis")
        print("=" * 50)
        
        # Analyze dataset structure
        self._analyze_structure()
        
        # Create sample visualization
        self._create_sample_grid()
        
        # Analyze image properties
        self._analyze_image_properties()
        
        # Save exploration results
        self._save_exploration_report()
        
        return self.dataset_stats
    
    def _analyze_structure(self):
        """Analyze dataset structure and count files"""
        print("🔍 Analyzing dataset structure...")
        
        total_images = 0
        for category, path in self.all_paths.items():
            if os.path.exists(path):
                image_files = [f for f in os.listdir(path) 
                             if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
                count = len(image_files)
                total_images += count
                self.dataset_stats[category] = {
                    'count': count,
                    'path': path,
                    'files': image_files[:10]  # Store first 10 filenames as samples
                }
                print(f"   {category}: {count} images")
            else:
                print(f"   ⚠️ {category}: Path not found - {path}")
                self.dataset_stats[category] = {'count': 0, 'path': path, 'files': []}
        
        print(f"\n📈 Total Images: {total_images}")
        self.dataset_stats['total_images'] = total_images
    
    def _create_sample_grid(self):
        """Create a grid showing sample images from each grade"""
        print("🖼️ Creating sample image grid...")
        
        fig, axes = plt.subplots(2, 4, figsize=(20, 10))
        fig.suptitle('Sample Images from Each Grade Category', fontsize=16, fontweight='bold')
        
        categories = list(self.all_paths.keys())
        sample_info = []
        
        for idx, category in enumerate(categories):
            row = idx // 4
            col = idx % 4
            
            path = self.all_paths[category]
            if os.path.exists(path):
                image_files = [f for f in os.listdir(path) 
                             if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
                
                if image_files:
                    # Select random sample
                    sample_file = random.choice(image_files)
                    sample_path = os.path.join(path, sample_file)
                    
                    try:
                        # Load and display image
                        img = cv2.imread(sample_path)
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        
                        axes[row, col].imshow(img_rgb)
                        axes[row, col].set_title(f"{category}\n({len(image_files)} images)", 
                                               fontsize=10, fontweight='bold')
                        axes[row, col].axis('off')
                        
                        # Store sample info
                        h, w = img.shape[:2]
                        sample_info.append({
                            'category': category,
                            'filename': sample_file,
                            'dimensions': f"{w}x{h}",
                            'file_size': os.path.getsize(sample_path) / 1024  # KB
                        })
                        
                    except Exception as e:
                        axes[row, col].text(0.5, 0.5, f"Error loading\n{category}", 
                                          ha='center', va='center')
                        axes[row, col].set_title(category)
                        axes[row, col].axis('off')
                else:
                    axes[row, col].text(0.5, 0.5, f"No images\nin {category}", 
                                      ha='center', va='center')
                    axes[row, col].set_title(category)
                    axes[row, col].axis('off')
            else:
                axes[row, col].text(0.5, 0.5, f"Path not found\n{category}", 
                                  ha='center', va='center')
                axes[row, col].set_title(category)
                axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.savefig('dataset_sample_grid.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save sample info
        self.dataset_stats['sample_info'] = sample_info
        print("✅ Sample grid saved as 'dataset_sample_grid.png'")
    
    def _analyze_image_properties(self):
        """Analyze image properties across the dataset"""
        print("📐 Analyzing image properties...")
        
        all_properties = []
        
        for category, path in self.all_paths.items():
            if os.path.exists(path) and self.dataset_stats[category]['count'] > 0:
                image_files = [f for f in os.listdir(path) 
                             if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
                
                # Sample up to 50 images per category for analysis
                sample_files = random.sample(image_files, min(50, len(image_files)))
                
                for file_name in tqdm(sample_files, desc=f"Analyzing {category}"):
                    try:
                        img_path = os.path.join(path, file_name)
                        img = cv2.imread(img_path)
                        
                        if img is not None:
                            h, w, c = img.shape
                            file_size = os.path.getsize(img_path) / 1024  # KB
                            
                            # Calculate basic statistics
                            mean_brightness = np.mean(img)
                            std_brightness = np.std(img)
                            
                            all_properties.append({
                                'category': category,
                                'filename': file_name,
                                'width': w,
                                'height': h,
                                'channels': c,
                                'aspect_ratio': w / h,
                                'total_pixels': w * h,
                                'file_size_kb': file_size,
                                'mean_brightness': mean_brightness,
                                'std_brightness': std_brightness
                            })
                    except Exception as e:
                        print(f"   ⚠️ Error analyzing {file_name}: {e}")
        
        # Convert to DataFrame for analysis
        df = pd.DataFrame(all_properties)
        
        # Generate summary statistics
        if len(df) > 0:
            summary_stats = {
                'total_analyzed': len(df),
                'avg_width': df['width'].mean(),
                'avg_height': df['height'].mean(),
                'avg_aspect_ratio': df['aspect_ratio'].mean(),
                'avg_file_size_kb': df['file_size_kb'].mean(),
                'width_range': (df['width'].min(), df['width'].max()),
                'height_range': (df['height'].min(), df['height'].max()),
                'aspect_ratio_range': (df['aspect_ratio'].min(), df['aspect_ratio'].max()),
                'common_dimensions': {f"{dims[0]}x{dims[1]}": count for dims, count in df.groupby(['width', 'height']).size().nlargest(5).items()}
            }
            
            self.dataset_stats['image_properties'] = summary_stats
            
            # Create property visualizations
            self._create_property_plots(df)
            
            print(f"📊 Analyzed {len(df)} images")
            print(f"   Average dimensions: {summary_stats['avg_width']:.0f}x{summary_stats['avg_height']:.0f}")
            print(f"   Average aspect ratio: {summary_stats['avg_aspect_ratio']:.2f}")
            print(f"   Average file size: {summary_stats['avg_file_size_kb']:.1f} KB")
        else:
            print("⚠️ No images found for property analysis")
    
    def _create_property_plots(self, df):
        """Create visualizations for image properties"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Dataset Image Properties Analysis', fontsize=16, fontweight='bold')
        
        # 1. Dimension distribution
        axes[0, 0].scatter(df['width'], df['height'], alpha=0.6, c=df['category'].astype('category').cat.codes)
        axes[0, 0].set_xlabel('Width (pixels)')
        axes[0, 0].set_ylabel('Height (pixels)')
        axes[0, 0].set_title('Image Dimensions Distribution')
        
        # 2. Aspect ratio distribution
        axes[0, 1].hist(df['aspect_ratio'], bins=30, alpha=0.7, edgecolor='black')
        axes[0, 1].set_xlabel('Aspect Ratio (W/H)')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].set_title('Aspect Ratio Distribution')
        axes[0, 1].axvline(1.0, color='red', linestyle='--', label='Square (1:1)')
        axes[0, 1].legend()
        
        # 3. File size distribution
        axes[0, 2].hist(df['file_size_kb'], bins=30, alpha=0.7, edgecolor='black')
        axes[0, 2].set_xlabel('File Size (KB)')
        axes[0, 2].set_ylabel('Frequency')
        axes[0, 2].set_title('File Size Distribution')
        
        # 4. Brightness distribution by category
        for category in df['category'].unique():
            subset = df[df['category'] == category]
            axes[1, 0].hist(subset['mean_brightness'], alpha=0.6, label=category, bins=20)
        axes[1, 0].set_xlabel('Mean Brightness')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].set_title('Brightness Distribution by Category')
        axes[1, 0].legend()
        
        # 5. Image count per category
        category_counts = df['category'].value_counts()
        axes[1, 1].bar(range(len(category_counts)), category_counts.values)
        axes[1, 1].set_xticks(range(len(category_counts)))
        axes[1, 1].set_xticklabels(category_counts.index, rotation=45, ha='right')
        axes[1, 1].set_ylabel('Number of Images Analyzed')
        axes[1, 1].set_title('Analyzed Images per Category')
        
        # 6. Aspect ratio by category
        df.boxplot(column='aspect_ratio', by='category', ax=axes[1, 2])
        axes[1, 2].set_title('Aspect Ratio by Category')
        axes[1, 2].set_xlabel('Category')
        axes[1, 2].set_ylabel('Aspect Ratio')
        
        plt.tight_layout()
        plt.savefig('dataset_properties_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✅ Property analysis plots saved as 'dataset_properties_analysis.png'")
    
    def _save_exploration_report(self):
        """Save comprehensive exploration report"""
        report = {
            'exploration_date': datetime.now().isoformat(),
            'dataset_structure': self.dataset_stats,
            'summary': {
                'total_categories': len(self.all_paths),
                'categories_with_data': sum(1 for stats in self.dataset_stats.values() 
                                          if isinstance(stats, dict) and stats.get('count', 0) > 0),
                'total_images': self.dataset_stats.get('total_images', 0)
            }
        }
        
        # Save as JSON
        with open('dataset_exploration_report.json', 'w') as f:
            json.dump(report, f, indent=2, default=str)
        
        # Save as readable text
        with open('dataset_exploration_report.txt', 'w') as f:
            f.write("DATASET EXPLORATION REPORT\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Exploration Date: {report['exploration_date']}\n\n")
            
            f.write("DATASET STRUCTURE:\n")
            f.write("-" * 20 + "\n")
            for category, stats in self.dataset_stats.items():
                if isinstance(stats, dict) and 'count' in stats:
                    f.write(f"{category}: {stats['count']} images\n")
            
            f.write(f"\nTOTAL IMAGES: {self.dataset_stats.get('total_images', 0)}\n\n")
            
            if 'image_properties' in self.dataset_stats:
                props = self.dataset_stats['image_properties']
                f.write("IMAGE PROPERTIES SUMMARY:\n")
                f.write("-" * 25 + "\n")
                f.write(f"Images Analyzed: {props['total_analyzed']}\n")
                f.write(f"Average Dimensions: {props['avg_width']:.0f}x{props['avg_height']:.0f}\n")
                f.write(f"Average Aspect Ratio: {props['avg_aspect_ratio']:.2f}\n")
                f.write(f"Average File Size: {props['avg_file_size_kb']:.1f} KB\n")
                f.write(f"Width Range: {props['width_range'][0]}-{props['width_range'][1]}\n")
                f.write(f"Height Range: {props['height_range'][0]}-{props['height_range'][1]}\n")
        
        print("✅ Exploration report saved:")
        print("   - dataset_exploration_report.json")
        print("   - dataset_exploration_report.txt")

# ==================== STEP 2: HIGH-QUALITY IMAGE PREPROCESSING ====================

def resize_with_padding(image, target_size, interpolation_method):
    """
    Resize image while maintaining aspect ratio using padding (letterboxing).
    This preserves the original image proportions without distortion.
    """
    h, w = image.shape[:2]
    target_w, target_h = target_size
    
    # Calculate scaling factor to fit image within target size
    scale = min(target_w / w, target_h / h)
    
    # Calculate new dimensions
    new_w = int(w * scale)
    new_h = int(h * scale)
    
    # Resize image using specified interpolation
    resized = cv2.resize(image, (new_w, new_h), interpolation=interpolation_method)
    
    # Create new image with target dimensions and fill with white background
    padded = np.full((target_h, target_w, 3), 255, dtype=np.uint8)
    
    # Calculate padding offsets to center the image
    y_offset = (target_h - new_h) // 2
    x_offset = (target_w - new_w) // 2
    
    # Place resized image in center of padded canvas
    padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized
    
    return padded

def process_and_zip_images(base_path="/kaggle/input/znz-cloves"):
    """
    Process all images with high-quality resizing and create organized zip files
    """
    print("📸 STEP 2: High-Quality Image Processing")
    print("=" * 50)
    
    # Define all paths including group photos
    all_folders = {
        "Grade 1": f"{base_path}/Grade 1",
        "Grade 2": f"{base_path}/Grade 2",
        "Grade 3": f"{base_path}/Grade 3", 
        "Grade 4": f"{base_path}/Grade 4",
        "Group_Grade_1": f"{base_path}/group_photos/grade1",
        "Group_Grade_2": f"{base_path}/group_photos/grade2",
        "Group_Grade_3": f"{base_path}/group_photos/grade3",
        "Group_Grade_4": f"{base_path}/group_photos/grade4"
    }
    
    # Create output directories
    output_base = "/kaggle/working/processed_images"
    os.makedirs(output_base, exist_ok=True)
    
    processing_log = []
    
    for folder_name, input_path in all_folders.items():
        if not os.path.exists(input_path):
            print(f"⚠️ Folder not found: {input_path}")
            continue
            
        # Get image files
        image_files = [f for f in os.listdir(input_path) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        
        if not image_files:
            print(f"⚠️ No images found in: {folder_name}")
            continue
        
        print(f"🔄 Processing {len(image_files)} images from {folder_name}")
        
        # Create output folders for different resolutions
        output_224 = os.path.join(output_base, f"{folder_name}_224x224")
        output_512 = os.path.join(output_base, f"{folder_name}_512x512")
        os.makedirs(output_224, exist_ok=True)
        os.makedirs(output_512, exist_ok=True)
        
        successful_224 = 0
        successful_512 = 0
        
        for i, file_name in enumerate(tqdm(image_files, desc=f"Processing {folder_name}")):
            try:
                # Load image
                img_path = os.path.join(input_path, file_name)
                img = cv2.imread(img_path)
                
                if img is None:
                    raise ValueError("Failed to load image")
                
                h, w = img.shape[:2]
                
                # Determine interpolation method based on scaling
                if max(h, w) > 224:
                    interp_224 = cv2.INTER_AREA
                else:
                    interp_224 = cv2.INTER_CUBIC
                
                if max(h, w) > 512:
                    interp_512 = cv2.INTER_AREA
                else:
                    interp_512 = cv2.INTER_CUBIC
                
                # Process both resolutions
                processed_224 = resize_with_padding(img, (224, 224), interp_224)
                processed_512 = resize_with_padding(img, (512, 512), interp_512)
                
                # Save processed images
                save_path_224 = os.path.join(output_224, file_name)
                save_path_512 = os.path.join(output_512, file_name)
                
                if cv2.imwrite(save_path_224, processed_224):
                    successful_224 += 1
                if cv2.imwrite(save_path_512, processed_512):
                    successful_512 += 1
                
            except Exception as e:
                print(f"   ❌ Error processing {file_name}: {e}")
        
        # Log results
        log_entry = {
            'folder': folder_name,
            'total_images': len(image_files),
            'successful_224': successful_224,
            'successful_512': successful_512,
            'success_rate_224': successful_224 / len(image_files),
            'success_rate_512': successful_512 / len(image_files)
        }
        processing_log.append(log_entry)
        
        print(f"   ✅ {folder_name}: {successful_224}/{len(image_files)} (224x224), {successful_512}/{len(image_files)} (512x512)")
    
    # Create zip files for download
    print("\n📦 Creating zip files for download...")
    
    zip_files = []
    
    # Zip by resolution
    for resolution in ['224x224', '512x512']:
        zip_name = f"processed_images_{resolution}.zip"
        zip_path = os.path.join("/kaggle/working", zip_name)
        
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for folder_name in all_folders.keys():
                folder_path = os.path.join(output_base, f"{folder_name}_{resolution}")
                if os.path.exists(folder_path):
                    for file_name in os.listdir(folder_path):
                        file_path = os.path.join(folder_path, file_name)
                        arcname = f"{folder_name}/{file_name}"
                        zipf.write(file_path, arcname)
        
        zip_files.append(zip_name)
        print(f"   ✅ Created: {zip_name}")
    
    # Save processing log
    log_df = pd.DataFrame(processing_log)
    log_df.to_csv('processing_log.csv', index=False)
    
    print(f"\n✅ Processing complete! Created {len(zip_files)} zip files:")
    for zip_file in zip_files:
        print(f"   📁 {zip_file}")
    
    return processing_log

# ==================== STEP 3: U-NET SEGMENTATION MODEL ====================

class UNet(nn.Module):
    """
    U-Net architecture for semantic segmentation
    Based on the original U-Net paper with modern improvements
    """
    def __init__(self, in_channels=3, out_channels=1, features=None):
        super(UNet, self).__init__()
        
        if features is None:
            features = [64, 128, 256, 512]
        
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList() 
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Down part of UNET
        in_ch = in_channels
        for feature in features:
            self.downs.append(DoubleConv(in_ch, feature))
            in_ch = feature
        
        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))
        
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
        
    def forward(self, x):
        skip_connections = []
        
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            
            if x.shape != skip_connection.shape:
                x = nn.functional.interpolate(x, size=skip_connection.shape[2:])
            
            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)
        
        return torch.sigmoid(self.final_conv(x))

class DoubleConv(nn.Module):
    """Double convolution block used in U-Net"""
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    
    def forward(self, x):
        return self.conv(x)

# ==================== STEP 4: CONTEXT-AWARE CLASSIFICATION MODEL ====================

class ContextAwareClassifier(nn.Module):
    """
    Classification model that uses both original image and segmentation mask
    This implements the "Combined Input" approach from recent research
    """
    def __init__(self, num_classes=4, backbone='resnet18', pretrained=True):
        super(ContextAwareClassifier, self).__init__()
        
        if backbone == 'resnet18':
            # Modify ResNet to accept 4 channels (RGB + mask)
            self.backbone = models.resnet18(pretrained=pretrained)
            # Replace first conv layer to accept 4 channels
            self.backbone.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)
        elif backbone == 'vgg16':
            self.backbone = models.vgg16(pretrained=pretrained)
            # Modify first layer for 4 channels
            first_layer = self.backbone.features[0]
            self.backbone.features[0] = nn.Conv2d(4, first_layer.out_channels, 
                                                 kernel_size=first_layer.kernel_size,
                                                 stride=first_layer.stride,
                                                 padding=first_layer.padding)
            self.backbone.classifier[6] = nn.Linear(4096, num_classes)
        
        # Initialize new weights for the additional channel
        self._init_new_channel_weights()
    
    def _init_new_channel_weights(self):
        """Initialize weights for the new mask channel"""
        if hasattr(self.backbone, 'conv1'):
            # For ResNet
            with torch.no_grad():
                # Copy RGB weights and average for mask channel
                old_weights = self.backbone.conv1.weight[:, :3, :, :].clone()
                self.backbone.conv1.weight[:, :3, :, :] = old_weights
                self.backbone.conv1.weight[:, 3:4, :, :] = old_weights.mean(dim=1, keepdim=True)
        else:
            # For VGG
            with torch.no_grad():
                old_weights = self.backbone.features[0].weight[:, :3, :, :].clone()
                self.backbone.features[0].weight[:, :3, :, :] = old_weights
                self.backbone.features[0].weight[:, 3:4, :, :] = old_weights.mean(dim=1, keepdim=True)
    
    def forward(self, x):
        return self.backbone(x)

# ==================== MAIN EXECUTION ====================

def run_context_aware_pipeline():
    """Execute the complete context-aware pipeline"""
    print("🚀 Starting Context-Aware Pipeline with Semantic Segmentation")
    print("=" * 70)
    
    # Step 1: Dataset Exploration
    explorer = DatasetExplorer()
    dataset_stats = explorer.explore_dataset()
    
    # Step 2: High-Quality Image Processing and Zipping
    processing_log = process_and_zip_images()
    
    print("\n🎯 PIPELINE STATUS:")
    print("=" * 30)
    print("✅ Dataset exploration complete")
    print("✅ High-quality image processing complete")
    print("✅ Zip files created for download")
    print("\n📁 OUTPUT FILES:")
    print("- dataset_sample_grid.png")
    print("- dataset_properties_analysis.png") 
    print("- dataset_exploration_report.json")
    print("- dataset_exploration_report.txt")
    print("- processed_images_224x224.zip")
    print("- processed_images_512x512.zip")
    print("- processing_log.csv")
    
    print("\n🔄 NEXT STEPS:")
    print("1. Download the zip files containing processed images")
    print("2. Create manual annotations for a subset of images (for segmentation training)")
    print("3. Train U-Net segmentation model on annotated data")
    print("4. Generate masks for full dataset")
    print("5. Train context-aware classifier with image+mask inputs")
    
    print(f"\n💾 All files saved to /kaggle/working/")
    
    return {
        'dataset_stats': dataset_stats,
        'processing_log': processing_log
    }

# ==================== STEP 5: ANNOTATION HELPER TOOLS ====================

class AnnotationHelper:
    """Helper tools for creating segmentation annotations"""
    
    def __init__(self, processed_images_path="/kaggle/working/processed_images"):
        self.processed_path = processed_images_path
        self.annotations_path = "/kaggle/working/annotations"
        os.makedirs(self.annotations_path, exist_ok=True)
    
    def create_annotation_template(self, num_samples_per_grade=5):
        """Create a structured annotation template for manual labeling"""
        print("🏷️ Creating Annotation Template")
        print("=" * 40)
        
        # Select representative samples from each grade
        annotation_plan = {}
        
        grade_folders = [f for f in os.listdir(self.processed_path) 
                        if f.endswith('224x224') and not f.startswith('Group_')]
        
        for folder_name in grade_folders:
            grade = folder_name.replace('_224x224', '')
            folder_path = os.path.join(self.processed_path, folder_name)
            
            if os.path.exists(folder_path):
                image_files = [f for f in os.listdir(folder_path) 
                             if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
                
                # Select diverse samples (by file size as proxy for complexity)
                if len(image_files) >= num_samples_per_grade:
                    # Get file sizes and select diverse samples
                    file_sizes = []
                    for img_file in image_files:
                        size = os.path.getsize(os.path.join(folder_path, img_file))
                        file_sizes.append((img_file, size))
                    
                    # Sort by size and select samples across the range
                    file_sizes.sort(key=lambda x: x[1])
                    indices = np.linspace(0, len(file_sizes)-1, num_samples_per_grade, dtype=int)
                    selected_files = [file_sizes[i][0] for i in indices]
                else:
                    selected_files = image_files[:num_samples_per_grade]
                
                annotation_plan[grade] = selected_files
        
        # Create annotation directory structure
        for grade, files in annotation_plan.items():
            grade_dir = os.path.join(self.annotations_path, grade)
            images_dir = os.path.join(grade_dir, 'images')
            masks_dir = os.path.join(grade_dir, 'masks')
            
            os.makedirs(images_dir, exist_ok=True)
            os.makedirs(masks_dir, exist_ok=True)
            
            # Copy selected images
            source_folder = os.path.join(self.processed_path, f"{grade}_224x224")
            for file_name in files:
                source_path = os.path.join(source_folder, file_name)
                dest_path = os.path.join(images_dir, file_name)
                shutil.copy2(source_path, dest_path)
                
                # Create empty mask template
                mask_name = file_name.replace('.jpg', '_mask.png').replace('.jpeg', '_mask.png')
                mask_path = os.path.join(masks_dir, mask_name)
                
                # Create blank mask (black image)
                blank_mask = np.zeros((224, 224), dtype=np.uint8)
                cv2.imwrite(mask_path, blank_mask)
        
        # Create annotation instructions
        instructions = """
ANNOTATION INSTRUCTIONS
======================

1. OBJECTIVE: Create pixel-level masks for clove objects in each image

2. TOOLS RECOMMENDED:
   - CVAT (Computer Vision Annotation Tool) - cvat.ai
   - LabelMe - github.com/wkentaro/labelme
   - Roboflow - roboflow.com
   - Or any image annotation software supporting segmentation

3. ANNOTATION GUIDELINES:
   - Mark the main clove object(s) in white (255)
   - Background should remain black (0)
   - Be precise around object boundaries
   - Include all visible parts of the clove
   - For group photos, annotate individual cloves separately if possible

4. FILE NAMING:
   - Keep image names unchanged
   - Mask files should be: [original_name]_mask.png
   - Masks should be 224x224 grayscale images

5. QUALITY CHECKS:
   - Mask should have clean edges
   - No gaps inside the object
   - No stray pixels outside the object
   - Check mask alignment with original image

6. DIRECTORY STRUCTURE:
   annotations/
   ├── Grade_1/
   │   ├── images/ (original images)
   │   └── masks/  (your annotations)
   ├── Grade_2/
   │   ├── images/
   │   └── masks/
   └── ... (etc)

Total images to annotate: {}

After annotation, use the train_segmentation_model() function to train U-Net.
        """.format(sum(len(files) for files in annotation_plan.values()))
        
        with open(os.path.join(self.annotations_path, 'ANNOTATION_INSTRUCTIONS.txt'), 'w') as f:
            f.write(instructions)
        
        # Save annotation plan
        plan_df = pd.DataFrame([(grade, len(files), files) for grade, files in annotation_plan.items()],
                              columns=['Grade', 'Number_of_Images', 'Selected_Files'])
        plan_df.to_csv(os.path.join(self.annotations_path, 'annotation_plan.csv'), index=False)
        
        print(f"✅ Annotation template created in: {self.annotations_path}")
        print(f"📊 Total images selected for annotation: {sum(len(files) for files in annotation_plan.values())}")
        for grade, files in annotation_plan.items():
            print(f"   {grade}: {len(files)} images")
        
        return annotation_plan

# ==================== STEP 6: SEGMENTATION TRAINING ====================

class SegmentationDataset(Dataset):
    """Dataset for training segmentation model"""
    
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        
        self.images = [f for f in os.listdir(images_dir) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.images_dir, img_name)
        
        # Corresponding mask
        mask_name = img_name.replace('.jpg', '_mask.png').replace('.jpeg', '_mask.png')
        mask_path = os.path.join(self.masks_dir, mask_name)
        
        # Load image and mask
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = mask.astype(np.float32) / 255.0  # Normalize to [0, 1]
        
        if self.transform:
            # Apply same transform to both image and mask
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
        # Convert to tensors
        image = transforms.ToTensor()(image)
        mask = torch.from_numpy(mask).unsqueeze(0)  # Add channel dimension
        
        return image, mask

def train_segmentation_model(annotations_path="/kaggle/working/annotations", 
                           num_epochs=50, batch_size=4, learning_rate=1e-4):
    """Train U-Net segmentation model on annotated data"""
    print("🧠 Training U-Net Segmentation Model")
    print("=" * 40)
    
    # Check if annotations exist
    if not os.path.exists(annotations_path):
        print("❌ Annotations not found! Please create annotations first using AnnotationHelper")
        return None
    
    # Collect all annotated data
    all_images_dirs = []
    all_masks_dirs = []
    
    for grade_folder in os.listdir(annotations_path):
        grade_path = os.path.join(annotations_path, grade_folder)
        if os.path.isdir(grade_path):
            images_dir = os.path.join(grade_path, 'images')
            masks_dir = os.path.join(grade_path, 'masks')
            
            if os.path.exists(images_dir) and os.path.exists(masks_dir):
                all_images_dirs.append(images_dir)
                all_masks_dirs.append(masks_dir)
    
    if not all_images_dirs:
        print("❌ No valid annotation directories found!")
        return None
    
    # Note: For demonstration, we'll create a simple training setup
    # In practice, you'd need proper augmentation library like albumentations
    print("⚠️  Note: This is a simplified training setup.")
    print("   For production, consider using albumentations for augmentation")
    
    # Initialize model
    model = UNet(in_channels=3, out_channels=1).to(device)
    
    # Loss and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print(f"✅ Model initialized on {device}")
    print(f"📊 Training parameters: epochs={num_epochs}, batch_size={batch_size}, lr={learning_rate}")
    
    # For this demo, we'll show the training structure
    training_structure = {
        'model': 'U-Net',
        'input_size': '224x224',
        'architecture': 'Encoder-Decoder with skip connections',
        'output': 'Binary segmentation mask',
        'loss_function': 'Binary Cross-Entropy',
        'optimizer': 'Adam',
        'data_augmentation': 'Recommended: rotation, flip, brightness, contrast'
    }
    
    print("🏗️ Training Structure:")
    for key, value in training_structure.items():
        print(f"   {key}: {value}")
    
    return model

# ==================== STEP 7: CONTEXT-AWARE DATASET ====================

class ContextAwareDataset(Dataset):
    """Dataset that combines images with segmentation masks for context-aware classification"""
    
    def __init__(self, images_dir, masks_dir, labels, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.labels = labels
        self.transform = transform
        
        self.image_files = [f for f in os.listdir(images_dir) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        
        # Load original image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load corresponding mask
        mask_name = img_name.replace('.jpg', '_predicted_mask.png').replace('.jpeg', '_predicted_mask.png')
        mask_path = os.path.join(self.masks_dir, mask_name)
        
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = mask.astype(np.float32) / 255.0
        else:
            # Fallback: create dummy mask if prediction doesn't exist
            mask = np.ones((224, 224), dtype=np.float32)
        
        if self.transform:
            image = self.transform(Image.fromarray(image))
        
        # Add mask as 4th channel
        mask_tensor = torch.from_numpy(mask).unsqueeze(0)
        combined_input = torch.cat([image, mask_tensor], dim=0)  # 4 channels: RGB + mask
        
        label = self.labels[idx]
        return combined_input, label

# ==================== STEP 8: COMPLETE PIPELINE EXECUTION ====================

def run_complete_context_aware_pipeline():
    """Run the complete context-aware pipeline with all steps"""
    
    results = run_context_aware_pipeline()
    
    # Create annotation helper
    print("\n🏷️ STEP 3: Setting up Annotation Framework")
    print("=" * 50)
    
    annotator = AnnotationHelper()
    annotation_plan = annotator.create_annotation_template(num_samples_per_grade=10)
    
    # Demonstrate segmentation training setup
    print("\n🧠 STEP 4: Segmentation Model Training Setup")
    print("=" * 50)
    
    segmentation_model = train_segmentation_model()
    
    print("\n🎯 CONTEXT-AWARE PIPELINE COMPLETION STATUS")
    print("=" * 55)
    print("✅ Dataset exploration and visualization complete")
    print("✅ High-quality image preprocessing complete") 
    print("✅ Zip files created for easy download")
    print("✅ Annotation framework and templates created")
    print("✅ U-Net segmentation model architecture ready")
    print("✅ Context-aware classifier architecture ready")
    
    print("\n📋 MANUAL STEPS REQUIRED:")
    print("1. 🖊️  Annotate the selected sample images (see annotations/ folder)")
    print("2. 🧠 Train U-Net on annotated data")
    print("3. 🔮 Generate masks for full dataset using trained U-Net")
    print("4. 🎯 Train context-aware classifier with image+mask inputs")
    print("5. 📊 Evaluate and compare with baseline models")
    
    print("\n💡 RESEARCH ADVANTAGES OF THIS APPROACH:")
    print("- Semantic understanding of object boundaries")
    print("- Robust to background variations")
    print("- Preserves fine-grained details for grading")
    print("- State-of-the-art approach for object classification")
    print("- Can handle complex backgrounds and lighting")
    
    print("\n📁 ALL OUTPUT FILES:")
    output_files = [
        "dataset_sample_grid.png",
        "dataset_properties_analysis.png",
        "dataset_exploration_report.json", 
        "dataset_exploration_report.txt",
        "processed_images_224x224.zip",
        "processed_images_512x512.zip",
        "processing_log.csv",
        "annotations/ (directory with annotation templates)",
        "annotations/ANNOTATION_INSTRUCTIONS.txt",
        "annotations/annotation_plan.csv"
    ]
    
    for file_name in output_files:
        print(f"   📄 {file_name}")
    
    print(f"\n🚀 Pipeline execution complete!")
    print(f"📊 Total dataset size: {results['dataset_stats'].get('total_images', 'Unknown')} images")
    print(f"🎯 Ready for manual annotation and model training phases")
    
    return results

# ==================== DEMO AND VISUALIZATION FUNCTIONS ====================

def create_pipeline_flowchart():
    """Create a visual flowchart of the context-aware pipeline"""
    
    fig, ax = plt.subplots(figsize=(14, 10))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 12)
    ax.axis('off')
    
    # Pipeline steps
    steps = [
        ("Raw Dataset\n(Individual + Group Photos)", 5, 11, 'lightblue'),
        ("Dataset Exploration\n& Visualization", 5, 9.5, 'lightgreen'), 
        ("High-Quality Preprocessing\n(Aspect Ratio + Padding)", 5, 8, 'lightyellow'),
        ("Manual Annotation\n(Subset of Images)", 2.5, 6.5, 'lightcoral'),
        ("U-Net Training\n(Segmentation)", 2.5, 5, 'lightpink'),
        ("Mask Generation\n(Full Dataset)", 2.5, 3.5, 'lightgray'),
        ("Baseline CNN\n(Original Images)", 7.5, 6.5, 'lightsteelblue'),
        ("Context-Aware CNN\n(Images + Masks)", 5, 2, 'gold'),
        ("Model Comparison\n& Evaluation", 5, 0.5, 'lightseagreen')
    ]
    
    # Draw boxes and text
    for text, x, y, color in steps:
        bbox = dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.7)
        ax.text(x, y, text, ha='center', va='center', fontsize=10, 
               fontweight='bold', bbox=bbox)
    
    # Draw arrows
    arrows = [
        (5, 10.7, 5, 9.8),  # Dataset -> Exploration
        (5, 9.2, 5, 8.3),   # Exploration -> Preprocessing
        (4.5, 7.7, 3, 6.8),  # Preprocessing -> Annotation
        (2.5, 6.2, 2.5, 5.3), # Annotation -> U-Net
        (2.5, 4.7, 2.5, 3.8), # U-Net -> Mask Generation
        (3, 3.5, 4.5, 2.3),  # Masks -> Context-Aware
        (5.5, 7.7, 7, 6.8),  # Preprocessing -> Baseline
        (6.5, 6.2, 5.5, 2.3), # Baseline -> Context-Aware
        (5, 1.7, 5, 0.8)     # Context-Aware -> Evaluation
    ]
    
    for x1, y1, x2, y2 in arrows:
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))
    
    plt.title('Context-Aware Pipeline Flowchart', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('context_aware_pipeline_flowchart.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Pipeline flowchart saved as 'context_aware_pipeline_flowchart.png'")

# Run the complete pipeline
if __name__ == "__main__":
    # Create pipeline visualization first
    create_pipeline_flowchart()
    
    # Run the complete pipeline
    results = run_complete_context_aware_pipeline()